In [1]:
import numpy as np
import os
import pandas as pd
import copy 
import datetime 
data_path = '../../../code_data/shanghai_t1dm/'
process_file = 'nips_23'

# pid_list = [1000 + d for d in range(1, 13)]


In [2]:
from xls2xlsx import XLS2XLSX
import shutil


for x in os.listdir(os.path.join(data_path, 'raw_data')):
    print(x)
    if not os.path.exists(os.path.join(data_path, 'processing/temp1')):
        os.makedirs(os.path.join(data_path, 'processing/temp1'))
    if x.endswith(".xls"):
        XLS2XLSX(os.path.join(data_path, 'raw_data', x)).to_xlsx(os.path.join(data_path, 'processing/temp1', x+'x'))
    elif x.endswith('.xlsx'):
        shutil.copyfile(os.path.join(data_path, 'raw_data', x), os.path.join(data_path, 'processing/temp1', x))

1001_0_20210730.xlsx
1002_0_20210504.xls
1002_1_20210521.xls
1002_2_20210909.xls
1003_0_20210831.xls
1004_0_20210425.xls
1005_0_20210522.xls
1006_0_20210114.xlsx
1006_1_20210209.xlsx
1006_2_20210303.xlsx
1007_0_20210726.xls
1008_0_20210713.xls
1009_0_20210803.xls
1010_0_20210915.xls
1011_0_20210622.xls
1012_0_20210923.xls


In [3]:
names = ['ts', 'glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',	'insulin_iv',] 
usecols = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]
pid2df = {}
for x in os.listdir(os.path.join(data_path,
                                  'processing/temp1')):
    if x.endswith('.xlsx'):
        # Prints only text file present in My Folder
        print(x)
        pid = int(x[:4])
        df = pd.read_excel(os.path.join(data_path, 'processing/temp1',x), names=names, usecols=usecols)
        if pid not in pid2df:
            pid2df[pid] = df
        else:
            pid2df[pid] = pd.concat([pid2df[pid], df])

1001_0_20210730.xlsx
1002_0_20210504.xlsx
1002_1_20210521.xlsx
1002_2_20210909.xlsx
1003_0_20210831.xlsx
1004_0_20210425.xlsx
1005_0_20210522.xlsx
1006_0_20210114.xlsx
1006_1_20210209.xlsx
1006_2_20210303.xlsx
1007_0_20210726.xlsx
1008_0_20210713.xlsx
1009_0_20210803.xlsx
1010_0_20210915.xlsx
1011_0_20210622.xlsx
1012_0_20210923.xlsx


In [4]:
import re
def change_meal(x):
    if pd.notna(x):
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        return np.array(result, dtype=float).sum()
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].apply(change_meal)

In [5]:
for pid in pid2df:
    pid2df[pid]['meal'] = pid2df[pid]['meal'].astype(np.float64)

In [6]:
def change_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+\s*IU')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan

        result = [res[:-3] for res in result]
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].apply(change_insulin)
    # pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_insulin)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].apply(change_insulin)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].apply(change_insulin)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].apply(change_insulin)

In [7]:
def nan_sum(x):
    if pd.notna(x[0]) and pd.notna(x[1]):
        return x[0] + x[1]
    elif pd.notna(x[0]):
        return x[0]
    elif pd.notna(x[1]):
        return x[1]
    else:
        return np.NaN

for pid in pid2df:
    pid2df[pid]['insulin_dose_sc'] = pid2df[pid]['insulin_dose_sc'].astype(np.float64)
    pid2df[pid]['insulin_bolus'] = pid2df[pid]['insulin_bolus'].astype(np.float64)
    pid2df[pid]['basal'] = pid2df[pid]['basal'].astype(np.float64)
    pid2df[pid]['insulin_iv'] = pid2df[pid]['insulin_iv'].astype(np.float64)
    pid2df[pid]['bolus'] = pid2df[pid][['insulin_dose_sc', 'insulin_bolus']].apply(nan_sum, axis=1)

In [8]:
def change_non_insulin(x):
    if pd.notna(x):
        if type(x) != str:
            return x
        pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')   # find_number
        result = pattern.findall(x)
        if len(result) == 0:
            return np.nan
        final = np.array(result, dtype=float).sum()
        return final
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].apply(change_non_insulin)


In [9]:
for pid in pid2df:
    pid2df[pid]['hypo_agents'] = pid2df[pid]['hypo_agents'].astype(np.float64)

In [10]:
pid2regular_df = {}
pid_list = list(pid2df.keys())
for pid in pid_list:
    df = pid2df[pid]
    df = df.set_index('ts')
    df = df.sort_index()
    pid2regular_df[pid] = df
    print(f'{pid}, total_points:{len(df)}, start:{df.index[0]}, end:{df.index[-1]}')


1001, total_points:658, start:2021-07-30 16:43:00, end:2021-08-06 12:58:00
1002, total_points:2238, start:2021-05-04 10:33:00, end:2021-09-13 09:06:00
1003, total_points:1339, start:2021-08-31 17:43:00, end:2021-09-14 16:13:00
1004, total_points:917, start:2021-04-25 17:26:00, end:2021-05-05 06:26:00
1005, total_points:1256, start:2021-05-22 11:59:00, end:2021-06-04 13:44:00
1006, total_points:3710, start:2021-01-14 18:39:00, end:2021-03-15 10:38:00
1007, total_points:1337, start:2021-07-26 09:58:00, end:2021-08-09 07:58:00
1008, total_points:766, start:2021-07-13 16:27:00, end:2021-07-21 15:42:00
1009, total_points:681, start:2021-08-03 12:04:00, end:2021-08-10 14:04:00
1010, total_points:918, start:2021-09-14 17:30:00, end:2021-09-24 06:45:00
1011, total_points:536, start:2021-06-22 17:23:00, end:2021-06-28 07:08:00
1012, total_points:1339, start:2021-09-23 01:08:00, end:2021-10-06 23:38:00


In [11]:
# check
for pid in pid_list:
    delta_ts = pid2regular_df[pid].index[1:] - pid2regular_df[pid].index[:-1]
    print(pid, (delta_ts>datetime.timedelta(minutes=15, seconds=10)).sum())

1001 0
1002 3
1003 0
1004 0
1005 0
1006 2
1007 0
1008 0
1009 0
1010 0
1011 0
1012 0


In [12]:
# add time features

def add_time_attributes(pid2data):
    for pid in pid2data:
        data = pid2data[pid]

        temp = data.index.to_frame().loc[:, 'ts'].dt
        
        data['day_of_week'] = temp.dayofweek
        data['day_of_week'] = data['day_of_week'].astype(np.float64)

        data['hour'] = temp.hour
        data['hour'] = data['hour'].astype(np.float64)

        data['minute'] = temp.minute
        data['minute'] = data['minute'].astype(np.float64)

        
        data['timestamp'] = temp.hour * 3600 +\
                                temp.minute * 60 +\
                                temp.second
        data['timestamp'] = data['timestamp'].astype(np.float64)
        
        # new ————————————————————————
        seconds_in_day = 24*60*60

        data['sin_time'] = np.sin(2 * np.pi * data.timestamp / seconds_in_day)
        data['cos_time'] = np.cos(2 * np.pi * data.timestamp / seconds_in_day)
        data['sin_time'].astype(np.float64)
        data['cos_time'].astype(np.float64)
        # end ______________________
        
        
    return pid2data

In [13]:
pid2regular_df = add_time_attributes(pid2regular_df)


In [14]:
pid2regular_df[1001].sin_time.describe()

count    658.000000
mean       0.013912
std        0.708048
min       -0.999962
25%       -0.700909
50%        0.032710
75%        0.713250
max        0.999962
Name: sin_time, dtype: float64

In [15]:
# add discrete y

def discrete(y):
    if y < 70:
        return 0
    elif 70<= y <=180:
        return 1
    elif y > 180:
        return 2
    else:
        return np.nan


def add_discrete_y(pid2data):
    for pid in pid2data:
        data = pid2data[pid]
        data['discrete_y'] = data['glucose_level']

        data['discrete_y'] = data['discrete_y'].apply(discrete)
        
    return pid2data

In [16]:
pid2regular_df = add_discrete_y(pid2regular_df)

In [17]:
pid2regular_df[1001].columns

Index(['glucose_level', 'finger_stick', 'blood_ketone', 'meal',
       'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',
       'insulin_iv', 'bolus', 'day_of_week', 'hour', 'minute', 'timestamp',
       'sin_time', 'cos_time', 'discrete_y'],
      dtype='object')

In [18]:
def add_indicate_for_nan(pid2data, ignore_indicate_set):
    for pid in pid2data:
        data = pid2data[pid]
        for attri in data:
            if attri in ignore_indicate_set:
                continue
            data[attri + '_indicate'] = 1
            data.loc[pd.notna(data[attri]), attri + '_indicate'] = 0
            if sum(data[attri + '_indicate']) != len(data[attri]) - sum(pd.notna(data[attri])):
                print('error')
    return pid2data

In [19]:
ignore_indicate_set = ['day_of_week', 'hour', 'minute',
       'timestamp', 'sin_time', 'cos_time',]
pid2regular_df = add_indicate_for_nan(pid2regular_df, ignore_indicate_set)

In [20]:
pid2regular_df[1001][['glucose_level', 'glucose_level_indicate']].iloc[560:]

,glucose_level,glucose_level_indicate
ts,,
2021-08-05 12:43:00,205.2,0
2021-08-05 12:58:00,194.4,0
2021-08-05 13:13:00,187.2,0
2021-08-05 13:28:00,194.4,0
2021-08-05 13:43:00,196.2,0
...,...,...
2021-08-06 11:58:00,279.0,0
2021-08-06 12:13:00,273.6,0
2021-08-06 12:28:00,266.4,0


In [21]:
pid2train = {}
pid2valid = {}
pid2test = {}

for pid in pid_list:
    idxs = pid2regular_df[pid].index[pd.notna(pid2regular_df[pid]['glucose_level'])]

    idxs = idxs.to_frame()

    total_len = len(idxs)
    train_end = int(total_len * 0.6)
    valid_end = int(total_len * 0.8) 

    pid2train[pid] = pid2regular_df[pid].loc[:idxs.iloc[train_end][0]]

    pid2valid[pid] = pid2regular_df[pid].loc[idxs.iloc[train_end + 1][0]:idxs.iloc[valid_end][0]]

    pid2test[pid] = pid2regular_df[pid].loc[idxs.iloc[valid_end + 1][0]:]

    print(pid, total_len, len(pid2train[pid]), len(pid2valid[pid]), len(pid2test[pid]))

1001 658 395 132 131
1002 2238 1343 448 447
1003 1339 804 268 267
1004 917 551 183 183
1005 1256 754 251 251
1006 3710 2227 742 741
1007 1337 803 267 267
1008 766 460 153 153
1009 681 409 136 136
1010 918 551 184 183
1011 536 322 107 107
1012 1339 804 268 267


In [22]:
# calculate pearson
df = pd.concat(list(pid2train.values()))
df[['glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus','bolus', 'basal',	'insulin_iv','timestamp']].corr(method ='pearson')['glucose_level'].sort_values()


insulin_bolus     -0.206449
basal             -0.054074
timestamp         -0.012223
bolus              0.066125
hypo_agents        0.074710
meal               0.234908
insulin_dose_sc    0.287520
insulin_iv         0.345652
blood_ketone       0.445147
finger_stick       0.908434
glucose_level      1.000000
Name: glucose_level, dtype: float64

In [23]:
for pid in pid2test:
    print(pid, 'blood_ketone', pd.notna(pid2test[pid]['blood_ketone']).sum())
    print(pid,'insulin_iv', pd.notna(pid2test[pid]['insulin_iv']).sum())

1001 blood_ketone 0
1001 insulin_iv 0
1002 blood_ketone 0
1002 insulin_iv 0
1003 blood_ketone 0
1003 insulin_iv 0
1004 blood_ketone 0
1004 insulin_iv 0
1005 blood_ketone 0
1005 insulin_iv 0
1006 blood_ketone 0
1006 insulin_iv 0
1007 blood_ketone 0
1007 insulin_iv 0
1008 blood_ketone 0
1008 insulin_iv 0
1009 blood_ketone 0
1009 insulin_iv 0
1010 blood_ketone 0
1010 insulin_iv 0
1011 blood_ketone 0
1011 insulin_iv 0
1012 blood_ketone 0
1012 insulin_iv 0


In [24]:
pid2train[pid].columns

Index(['glucose_level', 'finger_stick', 'blood_ketone', 'meal',
       'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'basal',
       'insulin_iv', 'bolus', 'day_of_week', 'hour', 'minute', 'timestamp',
       'sin_time', 'cos_time', 'discrete_y', 'glucose_level_indicate',
       'finger_stick_indicate', 'blood_ketone_indicate', 'meal_indicate',
       'insulin_dose_sc_indicate', 'hypo_agents_indicate',
       'insulin_bolus_indicate', 'basal_indicate', 'insulin_iv_indicate',
       'bolus_indicate', 'discrete_y_indicate'],
      dtype='object')

In [25]:
pid_attri2mean_std = {}
attris_should_norm = ['glucose_level', 'finger_stick', 'blood_ketone', 'meal', 'insulin_dose_sc', 'hypo_agents', 'insulin_bolus', 'bolus', 'basal',	'insulin_iv', 'timestamp']
for pid in pid_list:
    for attri in attris_should_norm:
        mean = pid2train[pid][attri].mean()
        std = pid2train[pid][attri].std()
        if std is np.NaN or std is pd.NaT or std == 0:
            std = 1e-6
        pid_attri2mean_std[(pid, attri)] = (mean, std)

if not os.path.exists(os.path.join(data_path, process_file)):
    os.makedirs(os.path.join(data_path, process_file))
np.save(os.path.join(data_path, process_file, 'pid_attri2mean_std.npy'), pid_attri2mean_std)

In [26]:
def norm_data(pid2data, pid_attri2mean_std, attris_should_norm):

    new_pid2data = {}
    for pid in pid2data:

        new_pid2data[pid] = copy.deepcopy(pid2data[pid]) 

        for attri in attris_should_norm:

            mean, std = pid_attri2mean_std[(pid, attri)]

            if std <= 1e-6:
                new_pid2data[pid][attri] = new_pid2data[pid][attri] / mean
            else:
                new_pid2data[pid][attri] = (new_pid2data[pid][attri] - mean) / std          

    return new_pid2data

In [27]:
pid2train_norm = norm_data(pid2train, pid_attri2mean_std, attris_should_norm)
pid2valid_norm = norm_data(pid2valid, pid_attri2mean_std, attris_should_norm)
pid2test_norm = norm_data(pid2test, pid_attri2mean_std, attris_should_norm)

In [28]:
attri_list_wo_glucose = [
    'finger_stick', 'blood_ketone', 'meal',
    'insulin_dose_sc', 'hypo_agents', 'insulin_bolus','bolus', 'basal',
    'insulin_iv', 'day_of_week', 'hour', 'minute', 'timestamp',
    'sin_time', 'cos_time', 'discrete_y', 'glucose_level_indicate',
    'finger_stick_indicate', 'blood_ketone_indicate', 'meal_indicate',
    'insulin_dose_sc_indicate', 'hypo_agents_indicate',
    'insulin_bolus_indicate', 'basal_indicate', 'insulin_iv_indicate',
]

temp = []

for idx, attri in enumerate(attri_list_wo_glucose):
    temp.append({'attri':attri, 'idx':idx}, )
attri2idx = pd.DataFrame(temp).set_index('attri')
idx2attri = pd.DataFrame(temp).set_index('idx')
print(attri2idx)
print(idx2attri)

save_path = os.path.join(data_path, process_file)

attri2idx.to_pickle(os.path.join(save_path,'attri2idx.pkl'))
idx2attri.to_pickle(os.path.join(save_path,'idx2attri.pkl'))


                          idx
attri                        
finger_stick                0
blood_ketone                1
meal                        2
insulin_dose_sc             3
hypo_agents                 4
insulin_bolus               5
bolus                       6
basal                       7
insulin_iv                  8
day_of_week                 9
hour                       10
minute                     11
timestamp                  12
sin_time                   13
cos_time                   14
discrete_y                 15
glucose_level_indicate     16
finger_stick_indicate      17
blood_ketone_indicate      18
meal_indicate              19
insulin_dose_sc_indicate   20
hypo_agents_indicate       21
insulin_bolus_indicate     22
basal_indicate             23
insulin_iv_indicate        24
                        attri
idx                          
0                finger_stick
1                blood_ketone
2                        meal
3             insulin_dose_sc
4         

In [29]:
for pid in pid_list:
    pid2train_norm[pid] = pid2train_norm[pid].fillna(0.0)
    pid2valid_norm[pid] = pid2valid_norm[pid].fillna(0.0)
    pid2test_norm[pid] = pid2test_norm[pid].fillna(0.0)

In [30]:
pid2train_norm[1001]

,glucose_level,finger_stick,blood_ketone,meal,insulin_dose_sc,hypo_agents,insulin_bolus,basal,insulin_iv,bolus,...,finger_stick_indicate,blood_ketone_indicate,meal_indicate,insulin_dose_sc_indicate,hypo_agents_indicate,insulin_bolus_indicate,basal_indicate,insulin_iv_indicate,bolus_indicate,discrete_y_indicate
ts,,,,,,,,,,,,,,,,,,,,,
2021-07-30 16:43:00,-1.770808,0.000000,0.0,0.000000,0.0,0.0,0.000000,-0.721688,0.0,0.000000,...,1,1,1,1,1,1,0,1,1,0
2021-07-30 16:58:00,-1.563250,0.000000,0.0,0.000000,0.0,0.0,-0.568708,0.000000,0.0,-0.568708,...,1,1,1,1,1,0,1,1,0,0
2021-07-30 17:13:00,-1.459471,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1,1,1,1,1,1,1,1,1,0
2021-07-30 17:28:00,-1.217321,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1,1,1,1,1,1,1,1,1,0
2021-07-30 17:43:00,-0.940577,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-03 18:13:00,-0.525462,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1,1,1,1,1,1,1,1,1,0
2021-08-03 18:28:00,-0.767613,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1,1,1,1,1,1,1,1,1,0
2021-08-03 18:43:00,-0.905984,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,1,1,1,1,1,1,1,1,1,0


In [31]:
# generate training data


def gen_final(pid2data, attri2idx,  n_prev, pred_window, name, valid_points = 24): 
    pid2data_npy = {}
    for pid in pid_list:
        data = pid2data[pid]
        
        y_list = []
        y_dis_list = []
        target_cos_time_list = []
        target_sin_time_list = []
        target_total_seconds_list = []
        attri_X_list = []
        attri_X_tar_list = []
        y_tar_list = []
        glucose_level_X_list = []
        
        seq_st_ed_list = []
        
        seq_st, seq_ed = None, None

        for idx in range(len(data) - n_prev - pred_window + 1):

            tm_idx = data.iloc[idx: idx + n_prev + pred_window].index
            tm_delta = (tm_idx[1:] - tm_idx[:-1]).total_seconds()/60/60
            tm_delta_max = tm_delta.max()
            if tm_delta_max > 2:
                if seq_st is not None and seq_ed - seq_st >= 2:
                    seq_st_ed_list.append([seq_st, seq_ed])
                seq_st, seq_ed = None, None
                continue

            glucose_level_y = data.iloc[idx + n_prev + pred_window - 1]['glucose_level']
            glucose_level_y_indicate = data.iloc[idx + n_prev + pred_window - 1]['glucose_level_indicate']
            if abs(glucose_level_y_indicate - 1) < 1e-6:
                continue

            # Resonable
            num_valids = int(valid_points - sum(data.iloc[idx + n_prev - valid_points : idx + n_prev]['glucose_level_indicate']))
            if num_valids == 0:
                if seq_st is not None and seq_ed - seq_st >= 2:
                    seq_st_ed_list.append([seq_st, seq_ed])
                seq_st, seq_ed = None, None
                continue
            # end

            
            y_list.append(glucose_level_y)
            y_dis_list.append(data.iloc[idx + n_prev + pred_window - 1]['discrete_y'])
            target_cos_time_list.append(data.iloc[idx + n_prev + pred_window - 1]['cos_time'])
            target_sin_time_list.append(data.iloc[idx + n_prev + pred_window - 1]['sin_time'])
            target_total_seconds_list.append(data.iloc[idx + n_prev + pred_window - 1]['timestamp'])
            glucose_level_X = np.array(data.iloc[idx: idx + n_prev]['glucose_level'].to_numpy(), dtype=np.float32)


            attri_X = np.array(data.iloc[idx: idx + n_prev][list(attri2idx.index)].to_numpy(), dtype=np.float32)          

            glucose_level_X_list.append(np.expand_dims(glucose_level_X, axis=0))

            attri_X_list.append(np.expand_dims(attri_X, axis=0))
            # new
            attri_X_tar = np.array(data.iloc[idx + n_prev: idx + n_prev + pred_window][list(attri2idx.index)].to_numpy(), dtype=np.float32)          
            attri_X_tar_list.append(np.expand_dims(attri_X_tar, axis=0))
            y_tar = data.iloc[idx + n_prev: idx + n_prev + pred_window]['glucose_level']
            y_tar_list.append(np.expand_dims(y_tar, axis=0)) 
            if seq_st is None:
                seq_st = len(y_list) - 1
            else:
                seq_ed = len(y_list) - 1
            
        if seq_st is not None and seq_ed - seq_st >= 2:
            seq_st_ed_list.append([seq_st, seq_ed])
        
        pid2data_npy[pid] = {}
        pid2data_npy[pid]['glucose_level_X'] = np.concatenate(glucose_level_X_list, axis=0)

        pid2data_npy[pid]['attri_X'] = np.concatenate(attri_X_list, axis=0)
        pid2data_npy[pid]['y'] = np.array(y_list, dtype=np.float32)
        pid2data_npy[pid]['y_dis'] = np.array(y_dis_list, dtype=np.int32)
        pid2data_npy[pid]['target_cos_time'] = np.array(target_cos_time_list, dtype=np.float32)
        pid2data_npy[pid]['target_sin_time'] = np.array(target_sin_time_list, dtype=np.float32)
        pid2data_npy[pid]['target_timestamp'] = np.array(target_total_seconds_list, dtype=np.float32)
        pid2data_npy[pid]['mean'] = pid_attri2mean_std[(pid, 'glucose_level')][0]
        pid2data_npy[pid]['std'] = pid_attri2mean_std[(pid, 'glucose_level')][1]
        pid2data_npy[pid]['attri_X_tar'] = np.concatenate(attri_X_tar_list, axis=0) 
        pid2data_npy[pid]['y_tar'] = np.concatenate(y_tar_list, axis=0)     
        pid2data_npy[pid]['seq_st_ed_list'] = np.array(seq_st_ed_list, dtype=np.int32)

        
        print(pid, pid2data_npy[pid]['glucose_level_X'].shape, pid2data_npy[pid]['attri_X'].shape)
        print('    ',len(y_list), seq_st_ed_list)
    save_path = os.path.join(data_path, process_file)
    np.save(os.path.join(save_path, f'{name}_{n_prev}_{pred_window}.npy'), pid2data_npy)




In [32]:
gen_final(pid2train_norm, attri2idx,  n_prev=16, pred_window=2, name='train_pid2data_npy', valid_points = 8)

1001 (378, 16) (378, 16, 25)
     378 [[0, 377]]
1002 (1309, 16) (1309, 16, 25)
     1309 [[0, 930], [931, 1308]]
1003 (787, 16) (787, 16, 25)
     787 [[0, 786]]
1004 (534, 16) (534, 16, 25)
     534 [[0, 533]]
1005 (737, 16) (737, 16, 25)
     737 [[0, 736]]
1006 (2193, 16) (2193, 16, 25)
     2193 [[0, 1218], [1219, 2192]]
1007 (786, 16) (786, 16, 25)
     786 [[0, 785]]
1008 (443, 16) (443, 16, 25)
     443 [[0, 442]]
1009 (392, 16) (392, 16, 25)
     392 [[0, 391]]
1010 (534, 16) (534, 16, 25)
     534 [[0, 533]]
1011 (305, 16) (305, 16, 25)
     305 [[0, 304]]
1012 (787, 16) (787, 16, 25)
     787 [[0, 786]]


In [33]:
gen_final(pid2train_norm, attri2idx,  n_prev=16, pred_window=4, name='train_pid2data_npy', valid_points = 8)

1001 (376, 16) (376, 16, 25)
     376 [[0, 375]]
1002 (1305, 16) (1305, 16, 25)
     1305 [[0, 928], [929, 1304]]
1003 (785, 16) (785, 16, 25)
     785 [[0, 784]]
1004 (532, 16) (532, 16, 25)
     532 [[0, 531]]
1005 (735, 16) (735, 16, 25)
     735 [[0, 734]]
1006 (2189, 16) (2189, 16, 25)
     2189 [[0, 1216], [1217, 2188]]
1007 (784, 16) (784, 16, 25)
     784 [[0, 783]]
1008 (441, 16) (441, 16, 25)
     441 [[0, 440]]
1009 (390, 16) (390, 16, 25)
     390 [[0, 389]]
1010 (532, 16) (532, 16, 25)
     532 [[0, 531]]
1011 (303, 16) (303, 16, 25)
     303 [[0, 302]]
1012 (785, 16) (785, 16, 25)
     785 [[0, 784]]


In [34]:
gen_final(pid2valid_norm, attri2idx,  n_prev=16, pred_window=2, name='valid_pid2data_npy', valid_points = 8)

1001 (115, 16) (115, 16, 25)
     115 [[0, 114]]
1002 (431, 16) (431, 16, 25)
     431 [[0, 430]]
1003 (251, 16) (251, 16, 25)
     251 [[0, 250]]
1004 (166, 16) (166, 16, 25)
     166 [[0, 165]]
1005 (234, 16) (234, 16, 25)
     234 [[0, 233]]
1006 (708, 16) (708, 16, 25)
     708 [[0, 330], [331, 707]]
1007 (250, 16) (250, 16, 25)
     250 [[0, 249]]
1008 (136, 16) (136, 16, 25)
     136 [[0, 135]]
1009 (119, 16) (119, 16, 25)
     119 [[0, 118]]
1010 (167, 16) (167, 16, 25)
     167 [[0, 166]]
1011 (90, 16) (90, 16, 25)
     90 [[0, 89]]
1012 (251, 16) (251, 16, 25)
     251 [[0, 250]]


In [35]:
gen_final(pid2valid_norm, attri2idx,  n_prev=16, pred_window=4, name='valid_pid2data_npy', valid_points = 8)

1001 (113, 16) (113, 16, 25)
     113 [[0, 112]]
1002 (429, 16) (429, 16, 25)
     429 [[0, 428]]
1003 (249, 16) (249, 16, 25)
     249 [[0, 248]]
1004 (164, 16) (164, 16, 25)
     164 [[0, 163]]
1005 (232, 16) (232, 16, 25)
     232 [[0, 231]]
1006 (704, 16) (704, 16, 25)
     704 [[0, 328], [329, 703]]
1007 (248, 16) (248, 16, 25)
     248 [[0, 247]]
1008 (134, 16) (134, 16, 25)
     134 [[0, 133]]
1009 (117, 16) (117, 16, 25)
     117 [[0, 116]]
1010 (165, 16) (165, 16, 25)
     165 [[0, 164]]
1011 (88, 16) (88, 16, 25)
     88 [[0, 87]]
1012 (249, 16) (249, 16, 25)
     249 [[0, 248]]


In [36]:
gen_final(pid2test_norm, attri2idx,  n_prev=16, pred_window=2, name='test_pid2data_npy', valid_points = 8)

1001 (114, 16) (114, 16, 25)
     114 [[0, 113]]
1002 (413, 16) (413, 16, 25)
     413 [[0, 72], [73, 412]]
1003 (250, 16) (250, 16, 25)
     250 [[0, 249]]
1004 (166, 16) (166, 16, 25)
     166 [[0, 165]]
1005 (234, 16) (234, 16, 25)
     234 [[0, 233]]
1006 (724, 16) (724, 16, 25)
     724 [[0, 723]]
1007 (250, 16) (250, 16, 25)
     250 [[0, 249]]
1008 (136, 16) (136, 16, 25)
     136 [[0, 135]]
1009 (119, 16) (119, 16, 25)
     119 [[0, 118]]
1010 (166, 16) (166, 16, 25)
     166 [[0, 165]]
1011 (90, 16) (90, 16, 25)
     90 [[0, 89]]
1012 (250, 16) (250, 16, 25)
     250 [[0, 249]]


In [37]:
gen_final(pid2test_norm, attri2idx,  n_prev=16, pred_window=4, name='test_pid2data_npy', valid_points = 8)

1001 (112, 16) (112, 16, 25)
     112 [[0, 111]]
1002 (409, 16) (409, 16, 25)
     409 [[0, 70], [71, 408]]
1003 (248, 16) (248, 16, 25)
     248 [[0, 247]]
1004 (164, 16) (164, 16, 25)
     164 [[0, 163]]
1005 (232, 16) (232, 16, 25)
     232 [[0, 231]]
1006 (722, 16) (722, 16, 25)
     722 [[0, 721]]
1007 (248, 16) (248, 16, 25)
     248 [[0, 247]]
1008 (134, 16) (134, 16, 25)
     134 [[0, 133]]
1009 (117, 16) (117, 16, 25)
     117 [[0, 116]]
1010 (164, 16) (164, 16, 25)
     164 [[0, 163]]
1011 (88, 16) (88, 16, 25)
     88 [[0, 87]]
1012 (248, 16) (248, 16, 25)
     248 [[0, 247]]
